Task:  control function approach (Petrin & Train, JMR) to account fot endogeneity with aggregate data

In [23]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import scipy.optimize as opt

#### Read in Data

In [24]:
df=pd.read_excel("CoffeeData.xlsx")
sjt=df["Share"]/df["Outside"]
df.head()

,Week,Share,Outside,Brand 1,Brand 2,Brand 3,Brand 4,Price,Feature,Display,F&D,Spot 1,Spot 2,Spot 3,Spot 4,Spot 5,Spot 6
0,1,0.005929,0.969501,1,0,0,0,4.29168,0.000000,0.000000,0.0,134,146,169,170,164,168
1,1,0.000732,0.969501,0,1,0,0,3.39696,0.000000,0.000000,0.0,134,146,169,170,164,168
2,1,0.016865,0.969501,0,0,1,0,2.93088,42.261265,0.000000,0.0,134,146,169,170,164,168
3,1,0.006973,0.969501,0,0,0,1,3.40096,0.000000,1.492933,0.0,134,146,169,170,164,168
4,2,0.006089,0.967560,1,0,0,0,4.29168,0.000000,0.000000,0.0,138,141,175,164,157,176


#### Step 1: Auxiliary regression

In [25]:
# First Stage: price on X_jt, intercepts, and instruments
X1=df[["Brand 1", "Brand 2", "Brand 3", "Brand 4",
       "Feature", "Display", "F&D",
       "Spot 1", "Spot 2", "Spot 3",
       "Spot 4", "Spot 5", "Spot 6"]]
resf=sm.OLS(df["Price"],X1).fit()
njt=df["Price"]-resf.predict(X1)

#### Step 2 & 3: Polynomials for njt

In [26]:
def cubic_poly(njt,a,b,c):
    return a*njt+b*np.power(njt,2)+c*np.power(njt,3)

In [27]:
#fn1t=cubic_poly(njt*df["Brand 1"],aa1,bb1,cc1)
#fn1t=cubic_poly(njt*df["Brand 2"],aa2,bb2,cc2)
#fn1t=cubic_poly(njt*df["Brand 3"],aa3,bb3,cc3)
#fn1t=cubic_poly(njt*df["Brand 4"],aa4,bb4,cc4)

#### Step 4: Starting values of $\alpha_j$,$\beta$,$aa_j$,$bb_j$,$cc_j$, j=1,...,4, and of $\Gamma$


In [28]:
D=30 #Number of random draws
np.random.seed(100)
draw=np.random.uniform(0,1,36).reshape(1,36)
a11,a12,a13,a14,a15,a16,a17,a18,b22,b23,b24,b25,b26,b27,b28,c33,c34,c35,c36,c37,c38,d44,d45,d46,d47,d48,e55,e56,e57,e58,f66,f67,f68,g77,f78,h88=draw[0]
r=np.matrix([[a11,a12,a13,a14,a15,a16,a17,a18], 
            [0,b22,b23,b24,b25,b26,b27,b28],
                [0,0,c33,c34,c35,c36,c37,c38],
                [0,0,0,d44,d45,d46,d47,d48],
                [0,0,0,0,e55,e56,e57,e58],
                [0,0,0,0,0,f66,f67,f68],
                [0,0,0,0,0,0,g77,f78],
                [0,0,0,0,0,0,0,h88]])
Q=np.random.normal(0,1,8*D).reshape(D,8)
a1,a2,a3,a4,bp,bf,bd,bfd=-2, -4, -3, -3, -1, 1, 1, -1
QGT=Q@r+np.array([a1,a2,a3,a4,bp,bf,bd,bfd])

#### Step 5: Compute $S_{jt}$

In [29]:
def crit(params, D):
    '''
    Function computes the non-linear least squares of Sjt (Computed share) and sjt (Actual Share)
    Input: params (initial values)
           D (number of draws)
    '''
    a1,a2,a3,a4,bp,bf,bd,bfd,aa1,bb1,cc1,aa2,bb2,cc2,aa3,bb3,cc3,aa4,bb4,cc4=params
    np.random.seed(100)
    Q=np.random.normal(0,1,8*D).reshape(D,8)
    QGT=Q@r+np.array([a1,a2,a3,a4,bp,bf,bd,bfd])  
    table=pd.DataFrame({"Week":df["Week"]})

    for i in range(D):
        #coef elements refer to a1,a2,a3,a4,bp,bf,bd,bfd
        coef=QGT[i].tolist()[0] #Get coefficients in the ith simulation
        ev1=coef[0]+coef[4]*df['Price']+coef[5]*df['Feature']+\
            coef[6]*df['Display']+coef[7]*df['F&D']+cubic_poly(njt*df["Brand 1"],aa1,bb1,cc1)
        ev2=coef[1]+coef[4]*df['Price']+coef[5]*df['Feature']+\
            coef[6]*df['Display']+coef[7]*df['F&D']+cubic_poly(njt*df["Brand 2"],aa2,bb2,cc2)
        ev3=coef[2]+coef[4]*df['Price']+coef[5]*df['Feature']+\
            coef[6]*df['Display']+coef[7]*df['F&D']+cubic_poly(njt*df["Brand 3"],aa3,bb3,cc3)
        ev4=coef[3]+coef[4]*df['Price']+coef[5]*df['Feature']+\
            coef[6]*df['Display']+coef[7]*df['F&D']+cubic_poly(njt*df["Brand 4"],aa4,bb4,cc4)
        summ=ev1+ev2+ev3+ev4
        table[i]=(ev1*df['Brand 1']+ev2*df['Brand 2']+ev3*df['Brand 3']+ev4*df['Brand 4'])/(1+summ)
    table=table.drop(columns=['Week'])
    Sjt=np.mean(table,axis=1)
    
    return np.sum(np.square(Sjt-sjt))

In [30]:
#Least square when initial parameters are 1 with 10 draws
crit(np.ones(20),10)

40.52370256360693

#### Step 6: Iterate over the values of $\alpha_j$,$\beta$,$aa_j$,$bb_j$,$cc_j$ till $S_{jt}\approx s_{jt}$

In [39]:
para_init=[-2.5, -4, -2.5, -3, -2, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
args=(10)
results = opt.minimize(crit, para_init,args)

In [40]:
#Minimized least squares
results.fun

23.806215018156664

In [41]:
#Standard Errors
se_list=[]
for i in range(8):
    se_list.append(np.sqrt(results.hess_inv[i,i]))

In [49]:
a1,a2,a3,a4,bp,bf,bd,bfd,aa1,bb1,cc1,aa2,bb2,cc2,aa3,bb3,cc3,aa4,bb4,cc4= results.x
tab=pd.DataFrame({"Control Function":[a1,a2,a3,a4,bp,bf,bd,bfd],
                  "CF-S.E":se_list,
                 "BLP":[-2.53249022, -4.4110098 , -2.28028224, -3.54245247, -0.21164297,
                         0.00457774,  0.00759952, -0.00515928],
                  "BLP-S.E":[0.28804858, 0.17166175, 0.18263546, 0.19311649, 0.0587026 ,
                         0.00070996, 0.00202677, 0.00260889]},
             index=["a1","a2","a3","a4","price","feature","display","F&D"])

In [51]:
tab.round(3)

,Control Function,CF-S.E,BLP,BLP-S.E
a1,-2.745,0.298,-2.532,0.288
a2,-2.689,0.742,-4.411,0.172
a3,-3.846,0.852,-2.280,0.183
a4,-3.966,0.596,-3.542,0.193
price,-1.810,0.032,-0.212,0.059
feature,0.315,0.002,0.005,0.001
display,0.282,0.003,0.008,0.002
F&D,-0.696,0.005,-0.005,0.003
